In [1]:
# Importar librerías necesarias
import pandas as pd
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense, Alias, Ord, Card
from copy import deepcopy
import gamspy


In [2]:
gamspy.__version__

'0.12.3'

In [2]:
def calcula_costo(todos_costos,costos_lugares_no_fue):
    print(costos_lugares_no_fue)
    print(todos_costos)
    suma_total = 0
    for lug in range(len(todos_costos)):
        suma_total = todos_costos[lug][1] + suma_total
    print('Costo total:', suma_total)
    suma_parcial = 0
    for lug in range(len(costos_lugares_no_fue)):
        suma_parcial = costos_lugares_no_fue[lug][1] + suma_parcial
    print('Costo que no fue :', suma_parcial)

    costo_acumulado=suma_total-suma_parcial
    
    print('Costo fue :', costo_acumulado)

    return costo_acumulado

In [3]:
# Definir el tiempo máximoo diario (minutos)
tiempo_max=550
# Definir el presupuesto máximo diario (minutos)
presupuesto_max=500
# Lista enumerada de 1 a total de lugares
lugares = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]

In [4]:
# Se crea una función
def ejecutar_proceso(tabla,tiempLug,tiempo_max,costosLug,presupuesto_max,numero_de_dias):

    # lugares = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    costo_acumulado=0
    todos_costos= costosLug.copy()
    costos_lugares_no_fue= costosLug.copy()
    for _ in range(numero_de_dias):

      m = Container()

      # Sets
      #lugares = [1,2,3,4,5,6]
      i = Set(container=m, name="i", description="punto i", records=lugares)
      j = Alias(m, name = "j", alias_with = i)
      k = Alias(m, name = "k", alias_with = i)
      #print(i.records)

      # Variables
      x = Variable(
          container=m,
          name="x",
          domain=[i, j],
          type="Binary",
          description="Si va del punto i al j",
      )
      y = Variable(
          container=m,
          name="y",
          domain=[i],
          type="Binary",
          description="Si pasa por el punto i",
      )
      u = Variable(
          container=m,
          name="u",
          domain=[i],
          type="Positive",
          description="Orden",
      )

      # Parámetros

      d = Parameter(
          container=m,
          name="d",
          domain=[i],
          description="parametro de tiempo en lugar",
          records=tiempLug,
      )

      p = Parameter(
          container=m,
          name="p",
          domain=[i],
          description="parametro de costos en el lugar",
          records=costosLug,
      )

      tiempos = Parameter(
          container=m,
          name="cT",
          domain=[i, j],
          description="tiempo que tarda en ir de i a j",
      )
      tiempos.setRecords(tabla.reset_index())
      #print(tiempos.records)

      # Ecuaciones
      objective = Sum((i, j), x[i,j])

      # r1(i)$(ord(i) = 1).. sum(j$(ord(j)>1), x(i,j)) =e= 1;
      r1 = Equation(
          container=m,
          name="r1",
          domain=[i],
          description="R Nodo Origen",
      )
      r1[i].where[Ord(i)==1]=Sum(j.where[Ord(j)>1], x[i, j]) == 1

      # r2(j)$(ord(j) = 1).. sum(i$(ord(i)>1), x(i,j)) =e= 1;
      r2 = Equation(
          container=m,
          name="r2",
          domain=[j],
          description="R Nodo Destino",
      )
      r2[j].where[Ord(j)==1]=Sum(i.where[Ord(i)>1], x[i, j]) == 1

      # r3(i).. sum(j,x(i,j))-sum(k,x(k,i))=e=0;
      r3 = Equation(
          container=m,
          name="r3",
          domain=[i],
          description="R Nodos intermedios",
      )
      r3[i]=Sum(j, x[i, j])-Sum(k, x[k, i]) == 0

      # r5(i)$(ord(i) > 1).. 2=l=u(i);
      r5 = Equation(
          container=m,
          name="r5",
          domain=[i],
          description="R Orden menor",
      )
      r5[i].where[Ord(i)>1]= u[i] >= 2

      # r6(i)$(ord(i) > 1).. u(i)=l=card(i);
      r6 = Equation(
          container=m,
          name="r6",
          domain=[i],
          description="R Orden mayor",
      )
      r6[i].where[Ord(i)>1]= u[i] <= Card(i)

      # r7(i,j)$(ord(i) > 1 and ord(j) > 1).. u(i)-u(j)+1=l=(card(i)-1)*(1-x(i,j));
      r7 = Equation(
          container=m,
          name="r7",
          domain=[i,j],
          description="R Ciclos",
      )
      r7[i,j].where[Ord(i)>1 and Ord(j)>1]= u[i]-u[j]+1 <= (Card(i)-1)*(1-x[i,j])

      # r8.. sum(i,sum(j,c(i,j)*x(i,j))) + sum(i,d(i)*y(i)) =l= 439;
      r8 = Equation(
          container=m,
          name="r8",
          description="R Tiempo",
      )
      r8[i,j]= Sum(i,Sum(j,tiempos[i,j]*x[i,j])) + Sum(i,d[i]*y[i]) <= tiempo_max

      # r9(i).. y(i) =g= sum(j, x(i,j));
      r9 = Equation(
          container=m,
          name="r9",
          domain=[i],
          description="R Visitar lugares",
      )
      r9[i]= y[i] >= Sum(j,x[i,j])

      #Restricción presupuesto

      r10 = Equation(
          container=m,
          name="r10",
          description="R Tiempo",
      )
      r10[i]= Sum(i,p[i]*y[i]) <= presupuesto_max -costo_acumulado



      # Se inicializa el modelo y resuelve
      elizalde = Model(
          container=m,
          name="elizalde",
          equations=m.getEquations(),
          problem="MIP",
          sense=Sense.MAX,
          objective=objective,
      )

      # Lo comentado sirve para visualizar información del modelo
      import sys
      elizalde.solve(output=sys.stdout)
      elizalde.solve()
      elizalde.objective_value

      # Se crea un DataFrame con las respuestas
      tax=x.records.set_index(["i", "j"])
      #print(tax['level'].items())
      nodos_interes = []

# Ciclo para encontrar los nodos con valor 1
      for indice, valor in tax['level'].items():
        if valor == 1:
            print("Nodos:")
            print(indice)
            nodos_interes.append((int(indice[0]), int(indice[1])))  # Guardar los pares (i, j)

# Ordenar los nodos según el camino que inicia y termina en 1
      conexiones = {inicio: fin for inicio, fin in nodos_interes}
      print("conexiones", conexiones)
      camino = []
      nodo_actual = 1

      while True:
        siguiente = conexiones.get(nodo_actual)
        if not siguiente:
            break  # Salimos si no hay más conexiones
        camino.append((nodo_actual, siguiente))
        nodo_actual = siguiente
        if nodo_actual == 1:
            break  # Si volvemos a 1, cerramos el ciclo

    # Imprimir el camino ordenado
      print("Camino ordenado:", camino)


      # Ciclo for para borrar en el DataFrame tabla y en tiempLug los nodos visitados
      for indice in camino:
        print(indice[1])
        numero_a_borrar = indice[1]
        numero_a_borrar2 = indice[0]

        if numero_a_borrar == 1:
            print("No lo borra")
        else:
            for i in lugares:
                tabla = tabla.drop((str(i), str(numero_a_borrar)))

            tabla = tabla.drop(index=str(numero_a_borrar))
            lugares.remove(numero_a_borrar)

            for xet in tiempLug:
                if xet[0] == str(numero_a_borrar):
                    valor3 = xet[1]

            tiempLug.remove([str(numero_a_borrar), valor3])

            for xet2 in costosLug:
                if xet2[0] == str(numero_a_borrar):
                    costo_lugar_visitado = xet2[1]


            costosLug.remove([str(numero_a_borrar), costo_lugar_visitado])  



            costos_lugares_no_fue=costosLug


            costo_dia=calcula_costo(todos_costos,costos_lugares_no_fue)


            todos_costos=costos_lugares_no_fue.copy()
            

            costo_acumulado= costo_acumulado+ costo_dia


    return elizalde

In [86]:
# DataFrame con las distancias entre cada lugar
tabla = pd.DataFrame(
    [
        ["1", "1", 99999],
        ["1", "2", 10],
        ["1", "3", 13],
        ["1", "4", 15],
        ["1", "5", 19],
        ["1", "6", 18],
        ["1", "7", 105],
        ["1", "8", 33],
        ["1", "9", 14],
        ["1", "10", 11],
        ["1", "11", 17],
        ["1", "12", 5],
        ["1", "13", 203],
        ["1", "14", 211],
        ["1", "15", 215],
        ["1", "16", 99],
        ["1", "17", 65],
        ["1", "18", 97],
        ["1", "19", 95],
        ["1", "20", 92],
        ["2", "1", 11],
        ["2", "2", 9999999],
        ["2", "3", 3],
        ["2", "4", 13],
        ["2", "5", 15],
        ["2", "6", 15],
        ["2", "7", 101],
        ["2", "8", 31],
        ["2", "9", 11],
        ["2", "10", 8],
        ["2", "11", 14],
        ["2", "12", 14],
        ["2", "13", 199],
        ["2", "14", 46],
        ["2", "15", 13],
        ["2", "16", 93],
        ["2", "17", 191],
        ["2", "18", 124],
        ["2", "19", 87],
        ["2", "20", 83],
        ["3", "1", 15],
        ["3", "2", 3],
        ["3", "3", 9999999],
        ["3", "4", 15],
        ["3", "5", 18],
        ["3", "6", 17],
        ["3", "7", 103],
        ["3", "8", 32],
        ["3", "9", 13],
        ["3", "10", 10],
        ["3", "11", 16],
        ["3", "12", 15],
        ["3", "13", 190],
        ["3", "14", 49],
        ["3", "15", 15],
        ["3", "16", 97],
        ["3", "17", 187],
        ["3", "18", 119],
        ["3", "19", 82],
        ["3", "20", 79],
        ["4", "1", 15],
        ["4", "2", 13],
        ["4", "3", 15],
        ["4", "4", 999999],
        ["4", "5", 3],
        ["4", "6", 2],
        ["4", "7", 107],
        ["4", "8", 36],
        ["4", "9", 16],
        ["4", "10", 14],
        ["4", "11", 19],
        ["4", "12", 20],
        ["4", "13", 205],
        ["4", "14", 54],
        ["4", "15", 20],
        ["4", "16", 127],
        ["4", "17", 202],
        ["4", "18", 132],
        ["4", "19", 102],
        ["4", "20", 93],
        ["5", "1", 20],
        ["5", "2", 13],
        ["5", "3", 15],
        ["5", "4", 1],
        ["5", "5", 999999999],
        ["5", "6", 20],
        ["5", "7", 106],
        ["5", "8", 36],
        ["5", "9", 16],
        ["5", "10", 14],
        ["5", "11", 19],
        ["5", "12", 20],
        ["5", "13", 202],
        ["5", "14", 51],
        ["5", "15", 23],
        ["5", "16", 135],
        ["5", "17", 202],
        ["5", "18", 132],
        ["5", "19", 100],
        ["5", "20", 93],
        ["6", "1", 18],
        ["6", "2", 17],
        ["6", "3", 19],
        ["6", "4", 4],
        ["6", "5", 4],
        ["6", "6", 99999999],
        ["6", "7", 110],
        ["6", "8", 40],
        ["6", "9", 20],
        ["6", "10", 18],
        ["6", "11", 23],
        ["6", "12", 24],
        ["6", "13", 206],
        ["6", "14", 55],
        ["6", "15", 22],
        ["6", "16", 130],
        ["6", "17", 205],
        ["6", "18", 134],
        ["6", "19", 103],
        ["6", "20", 100],
        ["7", "1", 110],
        ["7", "2", 97],
        ["7", "3", 103],
        ["7", "4", 131],
        ["7", "5", 134],
        ["7", "6", 134],
        ["7", "7", 9999999],
        ["7", "8", 704],
        ["7", "9", 94],
        ["7", "10", 95],
        ["7", "11", 87],
        ["7", "12", 101],
        ["7", "13", 212],
        ["7", "14", 31],
        ["7", "15", 112],
        ["7", "16", 31],
        ["7", "17", 195],
        ["7", "18", 129],
        ["7", "19", 108],
        ["7", "20", 101],
        ["8", "1", 33],
        ["8", "2", 33],
        ["8", "3", 35],
        ["8", "4", 37],
        ["8", "5", 39],
        ["8", "6", 39],
        ["8", "7", 75],
        ["8", "8", 9999999],
        ["8", "9", 32],
        ["8", "10", 32],
        ["8", "11", 24],
        ["8", "12", 46],
        ["8", "13", 221],
        ["8", "14", 17],
        ["8", "15", 39],
        ["8", "16", 69],
        ["8", "17", 195],
        ["8", "18", 153],
        ["8", "19", 118],
        ["8", "20", 118],
        ["9", "1", 15],
        ["9", "2", 12],
        ["9", "3", 14],
        ["9", "4", 12],
        ["9", "5", 14],
        ["9", "6", 14],
        ["9", "7", 96],
        ["9", "8", 24],
        ["9", "9", 999999],
        ["9", "10", 8],
        ["9", "11", 7],
        ["9", "12", 20],
        ["9", "13", 202],
        ["9", "14", 39],
        ["9", "15", 18],
        ["9", "16", 85],
        ["9", "17", 201],
        ["9", "18", 130],
        ["9", "19", 96],
        ["9", "20", 92],
        ["10", "1", 10],
        ["10", "2", 21],
        ["10", "3", 23],
        ["10", "4", 5],
        ["10", "5", 8],
        ["10", "6", 8],
        ["10", "7", 114],
        ["10", "8", 44],
        ["10", "9", 24],
        ["10", "10", 999999],
        ["10", "11", 27],
        ["10", "12", 28],
        ["10", "13", 211],
        ["10", "14", 59],
        ["10", "15", 16],
        ["10", "16", 88],
        ["10", "17", 209],
        ["10", "18", 139],
        ["10", "19", 107],
        ["10", "20", 104],
        ["11", "1", 17],
        ["11", "2", 13],
        ["11", "3", 15],
        ["11", "4", 14],
        ["11", "5", 8],
        ["11", "6", 17],
        ["11", "7", 89],
        ["11", "8", 21],
        ["11", "9", 8],
        ["11", "10", 9],
        ["11", "11", 9999999],
        ["11", "12", 21],
        ["11", "13", 204],
        ["11", "14", 36],
        ["11", "15", 21],
        ["11", "16", 81],
        ["11", "17", 202],
        ["11", "18", 131],
        ["11", "19", 100],
        ["11", "20", 92],
        ["12", "1", 5],
        ["12", "2", 13],
        ["12", "3", 16],
        ["12", "4", 20],
        ["12", "5", 23],
        ["12", "6", 23],
        ["12", "7", 109],
        ["12", "8", 38],
        ["12", "9", 19],
        ["12", "10", 16],
        ["12", "11", 22],
        ["12", "12", 9999999],
        ["12", "13", 204],
        ["12", "14", 53],
        ["12", "15", 105],
        ["12", "16", 105],
        ["12", "17", 203],
        ["12", "18", 132],
        ["12", "19", 108],
        ["12", "20", 93],
        ["13", "1", 202],
        ["13", "2", 191],
        ["13", "3", 198],
        ["13", "4", 275],
        ["13", "5", 278],
        ["13", "6", 273],
        ["13", "7", 201],
        ["13", "8", 219],
        ["13", "9", 201],
        ["13", "10", 199],
        ["13", "11", 204],
        ["13", "12", 204],
        ["13", "13", 999999],
        ["13", "14", 200],
        ["13", "15", 219],
        ["13", "16", 220],
        ["13", "17", 67],
        ["13", "18", 19],
        ["13", "19", 122],
        ["13", "20", 113],
        ["14", "1", 211],
        ["14", "2", 46],
        ["14", "3", 48],
        ["14", "4", 51],
        ["14", "5", 53],
        ["14", "6", 53],
        ["14", "7", 65],
        ["14", "8", 21],
        ["14", "9", 41],
        ["14", "10", 42],
        ["14", "11", 34],
        ["14", "12", 55],
        ["14", "13", 212],
        ["14", "14", 999999],
        ["14", "15", 66],
        ["14", "16", 61],
        ["14", "17", 210],
        ["14", "18", 141],
        ["14", "19", 114],
        ["14", "20", 117],
        ["15", "1", 216],
        ["15", "2", 105],
        ["15", "3", 107],
        ["15", "4", 193],
        ["15", "5", 196],
        ["15", "6", 196],
        ["15", "7", 3],
        ["15", "8", 78],
        ["15", "9", 61],
        ["15", "10", 61],
        ["15", "11", 91],
        ["15", "12", 174],
        ["15", "13", 201],
        ["15", "14", 109],
        ["15", "15", 999999],
        ["15", "16", 34],
        ["15", "17", 199],
        ["15", "18", 130],
        ["15", "19", 129],
        ["15", "20", 118],
        ["16", "1", 101],
        ["16", "2", 93],
        ["16", "3", 95],
        ["16", "4", 127],
        ["16", "5", 130],
        ["16", "6", 129],
        ["16", "7", 31],
        ["16", "8", 69],
        ["16", "9", 87],
        ["16", "10", 89],
        ["16", "11", 81],
        ["16", "12", 104],
        ["16", "13", 204],
        ["16", "14", 59],
        ["16", "15", 5],
        ["16", "16", 9999999],
        ["16", "17", 213],
        ["16", "18", 147],
        ["16", "19", 119],
        ["16", "20", 119],
        ["17", "1", 63],
        ["17", "2", 190],
        ["17", "3", 187],
        ["17", "4", 202],
        ["17", "5", 202],
        ["17", "6", 202],
        ["17", "7", 197],
        ["17", "8", 198],
        ["17", "9", 200],
        ["17", "10", 198],
        ["17", "11", 203],
        ["17", "12", 202],
        ["17", "13", 62],
        ["17", "14", 209],
        ["17", "15", 214],
        ["17", "16", 62],
        ["17", "17", 9999999],
        ["17", "18", 91],
        ["17", "19", 106],
        ["17", "20", 106],
        ["18", "1", 97],
        ["18", "2", 120],
        ["18", "3", 117],
        ["18", "4", 129],
        ["18", "5", 129],
        ["18", "6", 132],
        ["18", "7", 131],
        ["18", "8", 145],
        ["18", "9", 127],
        ["18", "10", 125],
        ["18", "11", 130],
        ["18", "12", 141],
        ["18", "13", 104],
        ["18", "14", 130],
        ["18", "15", 145],
        ["18", "16", 147],
        ["18", "17", 92],
        ["18", "18", 999999],
        ["18", "19", 45],
        ["18", "20", 41],
        ["19", "1", 121],
        ["19", "2", 79],
        ["19", "3", 77],
        ["19", "4", 105],
        ["19", "5", 97],
        ["19", "6", 102],
        ["19", "7", 101],
        ["19", "8", 113],
        ["19", "9", 88],
        ["19", "10", 86],
        ["19", "11", 91],
        ["19", "12", 106],
        ["19", "13", 102],
        ["19", "14", 103],
        ["19", "15", 121],
        ["19", "16", 123],
        ["19", "17", 113],
        ["19", "18", 44],
        ["19", "19", 999999],
        ["19", "20", 2],
        ["20", "1", 80],
        ["20", "2", 78],
        ["20", "3", 114],
        ["20", "4", 114],
        ["20", "5", 117],
        ["20", "6", 117],
        ["20", "7", 104],
        ["20", "8", 113],
        ["20", "9", 89],
        ["20", "10", 87],
        ["20", "11", 92],
        ["20", "12", 93],
        ["20", "13", 121],
        ["20", "14", 87],
        ["20", "15", 123],
        ["20", "16", 125],
        ["20", "17", 113],
        ["20", "18", 46],
        ["20", "19", 4],
        ["20", "20", 999999]
    ],
    columns=["from", "to", "distance"],
).set_index(["from", "to"])

# Lista con los tiempos de visita
tiempLug = [['1',0],['2',60],['3',60],['4',120],['5',120],['6',120],['7',120],['8',120],['9',120],['10',60],['11',60],['12',45],['13',90],['14',300],['15',120],['16',90],['17',60],['18',120],['19',60],['20',60]]

costosLug = [['1',1],['2',60],['3',60],['4',120],['5',120],['6',120],['7',120],['8',120],['9',120],['10',60],['11',60],['12',45],['13',90],['14',300],['15',120],['16',90],['17',60],['18',120],['19',60],['20',60]]



tabla_modificada = ejecutar_proceso(tabla,tiempLug,tiempo_max,costosLug,presupuesto_max,numero_de_dias=3)


[['1', 1], ['2', 60], ['3', 60], ['4', 120], ['5', 120], ['6', 120], ['7', 120], ['8', 120], ['9', 120], ['10', 60], ['11', 60], ['12', 45], ['13', 90], ['14', 300], ['15', 120], ['16', 90], ['17', 60], ['18', 120], ['19', 60], ['20', 60]]
--- Job _job_098d301c-1524-4908-b827-154eb269be08.gms Start 10/21/24 16:19:27 47.6.0 c2de9d6d WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Gael\Documents\Lib\site-packages\gamspy_base\gmsprmNT.txt
--- GAMS Parameters defined
    LP CPLEX
    MIP CPLEX
    RMIP CPLEX
    NLP IPOPTH
    MCP PATH
    MPEC NLPEC
    RMPEC CONVERT
    CNS PATH
    DNLP IPOPTH
    RMINLP IPOPTH
    MINLP SHOT
    QCP IPOPTH
    MIQCP SHOT
    RMIQCP IPOPTH
    EMP CONVERT
    Restart C:\Users\Gael\AppData\Local\Temp\tmp7z4j4g70\_restart_359f67d4-288b-4cb5-a3c2-dec20efb3ba6.g00
    Input C:\Users\Gael\AppData\Local\Temp\tmp7z4j4g70\_job_098d301c-1524-4908-b827-154eb269be08.gms
    Output C:\Users\Gael\AppData\Local\Temp\tmp7z4j4g70\_job_098d301c-1524-4908-b827-15

C:\Users\Gael\AppData\Local\Temp\ipykernel_7432\1281661945.py:215: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  tabla = tabla.drop(index=str(numero_a_borrar))


--- Job _job_1cf941d0-3f19-4b95-9962-f1658ee03f18.gms Start 10/21/24 16:19:35 47.6.0 c2de9d6d WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Gael\Documents\Lib\site-packages\gamspy_base\gmsprmNT.txt
--- GAMS Parameters defined
    LP CPLEX
    MIP CPLEX
    RMIP CPLEX
    NLP IPOPTH
    MCP PATH
    MPEC NLPEC
    RMPEC CONVERT
    CNS PATH
    DNLP IPOPTH
    RMINLP IPOPTH
    MINLP SHOT
    QCP IPOPTH
    MIQCP SHOT
    RMIQCP IPOPTH
    EMP CONVERT
    Restart C:\Users\Gael\AppData\Local\Temp\tmp98tmbysf\_restart_3410b8c3-0f18-410a-a54f-b57878429c9f.g00
    Input C:\Users\Gael\AppData\Local\Temp\tmp98tmbysf\_job_1cf941d0-3f19-4b95-9962-f1658ee03f18.gms
    Output C:\Users\Gael\AppData\Local\Temp\tmp98tmbysf\_job_1cf941d0-3f19-4b95-9962-f1658ee03f18.lst
    Save C:\Users\Gael\AppData\Local\Temp\tmp98tmbysf\_save_3410b8c3-0f18-410a-a54f-b57878429c9f.g00
    ScrDir C:\Users\Gael\AppData\Local\Temp\tmp98tmbysf\225a\
    SysDir C:\Users\Gael\Documents\Lib\site-packages\gamspy_ba

C:\Users\Gael\AppData\Local\Temp\ipykernel_7432\1281661945.py:215: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  tabla = tabla.drop(index=str(numero_a_borrar))


--- Job _job_e9647187-9c99-4fef-9be8-79b2e1ccb53a.gms Start 10/21/24 16:19:42 47.6.0 c2de9d6d WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Gael\Documents\Lib\site-packages\gamspy_base\gmsprmNT.txt
--- GAMS Parameters defined
    LP CPLEX
    MIP CPLEX
    RMIP CPLEX
    NLP IPOPTH
    MCP PATH
    MPEC NLPEC
    RMPEC CONVERT
    CNS PATH
    DNLP IPOPTH
    RMINLP IPOPTH
    MINLP SHOT
    QCP IPOPTH
    MIQCP SHOT
    RMIQCP IPOPTH
    EMP CONVERT
    Restart C:\Users\Gael\AppData\Local\Temp\tmptz52_q19\_restart_62da264d-e81e-449b-991d-46fe1165057c.g00
    Input C:\Users\Gael\AppData\Local\Temp\tmptz52_q19\_job_e9647187-9c99-4fef-9be8-79b2e1ccb53a.gms
    Output C:\Users\Gael\AppData\Local\Temp\tmptz52_q19\_job_e9647187-9c99-4fef-9be8-79b2e1ccb53a.lst
    Save C:\Users\Gael\AppData\Local\Temp\tmptz52_q19\_save_62da264d-e81e-449b-991d-46fe1165057c.g00
    ScrDir C:\Users\Gael\AppData\Local\Temp\tmptz52_q19\225a\
    SysDir C:\Users\Gael\Documents\Lib\site-packages\gamspy_ba

AttributeError: 'NoneType' object has no attribute 'set_index'

In [ ]:
nodos_interes = [(1,2),(2,10),(4,1),(10,4)]
conexiones = {inicio: fin for inicio, fin in nodos_interes}
print(conexiones)

camino = []
nodo_actual = 1

while True:
    siguiente = conexiones.get(nodo_actual)
    if not siguiente:
        break  # Salimos si no hay más conexiones
    camino.append((nodo_actual, siguiente))
    nodo_actual = siguiente
    if nodo_actual == 1:
        break  # Si volvemos a 1, cerramos el ciclo

# Imprimir el camino ordenado
print("Camino ordenado:", camino)